In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, scale
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn import metrics


%matplotlib inline

In [2]:
data = pd.read_csv('train.csv')

In [3]:
new_data = pd.DataFrame()

new_data['initial_list_status'] = data['initial_list_status'].map({'f': 0, 'w': 1})
new_data['dti'] = data['dti']
new_data['int_rate'] = data['int_rate']
new_data['emp_length'] = data['emp_length'].map({'< 1 year': 1, '1 year': 2, '2 years': 3,  '3 years': 4,  '4 years': 5,  '5 years': 6,  '6 years': 7,  '7 years': 8,  '8 years': 9,  '9 years': 10,  '10+ years': 11})
new_data['emp_length'].fillna(0, inplace=True)

new_data['mths_since_last_delinq'] = data['mths_since_last_delinq']
new_data['mths_since_last_delinq'].fillna(0, inplace=True)

In [4]:
X = new_data
X_train, X_test = train_test_split(X, test_size=0.3, random_state=42)
X_train = scale(X_train)
X_test = scale(X_test)

C:\Users\NikitsinskayaH\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by the scale function.
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\NikitsinskayaH\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by the scale function.
  after removing the cwd from sys.path.


In [6]:
from sklearn.cluster import KMeans


kmeans = KMeans(n_clusters=4, random_state=32)
preds_train = kmeans.fit_predict(X_train)
  
print(preds_train)

[0 2 1 ... 0 0 2]


In [7]:
preds_test = kmeans.predict(X_test)
print(preds_test)

[3 0 2 ... 1 3 2]


In [8]:
from sklearn.tree import DecisionTreeClassifier


reg_tree = DecisionTreeClassifier(random_state=42)

reg_tree.fit(X_train, preds_train)

preds = reg_tree.predict(X_test)
print(preds)

[3 0 2 ... 1 3 2]


In [9]:
print('Tree: ')
print('Accuracy score: ' + str(metrics.accuracy_score(preds_test, preds)))

print('Recall score: ' + str(metrics.recall_score(preds_test, preds, average='micro')))

print('Precision score: ' + str(metrics.precision_score(preds_test, preds, average='micro')))

Tree: 
Accuracy score: 0.9942221556188288
Recall score: 0.9942221556188288
Precision score: 0.9942221556188288
